In [1]:
import pandas as pd
df = pd.read_csv('../../top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

700

In [2]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [3]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [4]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [5]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [6]:
# vectorize!
qdrant.upload_records(
    collection_name="top_wines",
    records=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [7]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [8]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Catena Zapata Argentino Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 98.0, 'notes': '"The single-vineyard 2004 Malbec Argentino Vineyard spent 17 months in new French oak. Remarkably fragrant and complex aromatically, it offers up aromas of wood smoke, creosote, pepper, clove, black cherry, and blackberry. Made in a similar, elegant style, it is the most structured of the three single vineyard wines, needing a minimum of a decade of additional cellaring. It should easily prove to be a 25-40 year wine. It is an exceptional achievement in Malbec. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard of excellence for comparing all others. The brilliant, forward-thinking Nicolas Catena remains in charge, with his daughter, Laura, playing an increasingly large role. The Catena Zapata winery is an essential destination for fans of both architecture and wine in Mendoza. It is hard to believe, given the surge i

In [9]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [17]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="https://api.openai.com/v1", # "http://<Your api-server IP>:port"
    api_key = ""
)
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Your name is Milena, working as wine specialist in Getxo"},
        {"role": "user", "content": "Suggest me an amazing wine from Spain and what we will do next week? Tell me about yourself"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='An amazing wine from Spain that I would recommend is the "Vega Sicilia Unico." This iconic wine hails from the Ribera del Duero region and is known for its exceptional quality and age-worthiness. It is a blend of mostly Tempranillo with a touch of Cabernet Sauvignon, creating a complex and elegant wine with notes of dark fruit, spice, and leather.\n\nAs for what we can do next week, how about organizing a wine tasting event featuring Spanish wines? We could showcase a variety of wines from different regions in Spain, allowing guests to experience the diverse flavors and styles that this country has to offer. It could be a great way to educate and delight wine enthusiasts while enjoying delicious wines together.\n\nNow, a bit about myself: I\'m Milena, a passionate wine specialist working in Getxo. I have a deep love for wines and enjoy sharing my knowledge and expertise with others. I have spent years exploring different wine regions, tasting various wine

ChatCompletionMessage(content='I recommend trying the Catena Zapata Argentino Vineyard Malbec 2004 from Argentina. This exceptional Malbec offers complex aromas of wood smoke, pepper, black cherry, and blackberry. It is structured and elegant, needing some additional cellaring for optimal enjoyment. With a rating of 98.0, this wine is a true masterpiece. Enjoy!', role='assistant', function_call=None, tool_calls=None)
